In [ ]:
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DataType
from pyspark.sql.functions import col

In [ ]:
config = {"fs.azure.account.auth.type":"OAuth",
"fs.azure.account.oauth.provider.type":"org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id":"<clientid>",
"fs.azure.account.oauth2.client.secret":'<clien_secret>',
"fs.azure.account.oauth2.client.endpoint":"https://login.microsoftonline.com/<app_registration>/oauth2/token"
}

dbutils.fs.mount(
    source = "abfss://tokyoolympicsdata@tokyoolympicsdataraj.dfs.core.windows.net", # container@storageaccount
    mount_point = "/mnt/tokyoolympics",
    extra_configs = config)

Out[4]: True

In [ ]:
%fs
ls "/mnt/tokyoolympics"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympics/raw_data/,raw_data/,0,1721912431000
dbfs:/mnt/tokyoolympics/transformed_data/,transformed_data/,0,1721912446000


In [ ]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [ ]:
athletes = spark.read.csv("/mnt/tokyoolympics/raw_data/athletes.csv", header=True)
coaches = spark.read.csv("/mnt/tokyoolympics/raw_data/coaches.csv", header=True)
teams = spark.read.csv("/mnt/tokyoolympics/raw_data/teams.csv", header=True)
entries_gender = spark.read.csv("/mnt/tokyoolympics/raw_data/entries_gender.csv", header=True,inferSchema=True)
medals = spark.read.csv("/mnt/tokyoolympics/raw_data/medals.csv", header=True,inferSchema=True)

In [ ]:
athletes.show()

+--------------------+--------------------+-------------------+
|                Name|                 NOC|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [ ]:
athletes.printSchema()

root
 |-- Name: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [ ]:
entries_gender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [ ]:
entries_gender = entries_gender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
        .withColumn("Total",col("Total").cast(IntegerType()))

In [ ]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team/NOC: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [ ]:
#countries with most gold
top_countries_by_gold = medals.orderBy("Gold", ascending=False).show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|            Team/NOC|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [ ]:
athletes.repartition(1).write.mode("overwrite").csv("/mnt/tokyoolympics/transformed_data/athletes.csv", header=True)
coaches.repartition(1).write.mode("overwrite").csv("/mnt/tokyoolympics/transformed_data/coaches.csv", header=True)
entries_gender.repartition(1).write.mode("overwrite").csv("/mnt/tokyoolympics/transformed_data/entries_gender.csv", header=True)
teams.repartition(1).write.mode("overwrite").csv("/mnt/tokyoolympics/transformed_data/teams.csv", header=True)
medals.repartition(1).write.mode("overwrite").csv("/mnt/tokyoolympics/transformed_data/medals.csv", header=True)